In [ ]:
import numpy as np

In [ ]:
class NumberLineSystem:
    """
    1D particle system with collision and potential field.

    State: x[t] = [y[t], v[t]]
    Input: u[t] ∈ [-1, 1]
    Output: z[t] = y[t] + measurement noise

    Dynamics:
    v[t+1] = 0           with probability p_c * |v[t]|/v_max
    v[t+1] = v[t] + (u[t] + f_φ(y[t]))/m + N(0, 0.01*v[t]^2) otherwise

    y[t+1] = y[t] + v[t+1] + N(0, 0.25*v[t+1]^2)
    """

    def __init__(self, m=1.0, p_c=0.1, v_max=5.0, y0=0.0, v0=0.0, phi_y=None):
        self.m = m
        self.p_c = p_c
        self.v_max = v_max
        self.y = y0
        self.v = v0
        self.phi_y = 0
        self.history = {'y': [y0], 'v': [v0], 'z': [y0]}

    def f_phi(self, y):
        self.phi_y = np.sin(y)
        return self.phi_y

    def step(self, u):
        u = np.clip(u, -1, 1)
        collision_prob = self.p_c * abs(self.v) / self.v_max
        if np.random.random() < collision_prob:
            v_new = 0.0
        else:
            noise_v = np.random.normal(0, np.sqrt(0.01 * self.v ** 2))
            v_new = self.v + (u + self.f_phi(self.y)) / self.m + noise_v
        noise_y = np.random.normal(0, np.sqrt(0.25 * v_new ** 2))
        y_new = self.y + v_new + noise_y
        z = y_new + np.random.normal(0, 0.05)
        self.y, self.v = y_new, v_new
        self.history['y'].append(self.y)
        self.history['v'].append(self.v)
        self.history['z'].append(z)
        return z

    def simulate(self, inputs):
        return [self.step(u) for u in inputs]
